# tarefas
## aplicas strip em todas colunas
## retirar "Total" e "NaN"

In [23]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
class Scraper():
    def __init__(self):
        self.base_url = 'http://vitibrasil.cnpuv.embrapa.br/index.php'
        self.df = None
        self.max_year = None
        self.min_year = None
        self.anos = [ano for ano in range(self.min_year, self.max_year)]

    def get_year(self):
        response = requests.get(self.base_url + '?opcao=opt_02')
        soup = BeautifulSoup(response.content, 'html.parser')
        select_anos = soup.find('input', {'class':'text_pesq'})
        self.ano_max = select_anos['max']
        self.ano_min = ano_min = select_anos['min']
        
    def categorizar(self, df: pd.DataFrame):
        categoria_atual = None
        categorias = []

        for produto in df['Produto']:
            produto_str = str(produto).strip()

            if produto_str.isupper() and produto_str != 'NAN':
                categoria_atual = produto_str

            categorias.append(categoria_atual)

        df['Categoria'] = categorias
        return df

    
    def remover_categorias(self, df):
        mask = df['Produto'].apply(lambda x: isinstance(x, str) and not x.strip().isupper())
        return df[mask].reset_index(drop=True)
    
    def table_producao(self):
        dfs = []
        for ano in self.anos:
            url = f"{self.base_url}?ano={ano}&opcao=opt_02"
            try:
                df_ano = pd.read_html(url)[3]
                df_ano['ano'] = ano
                dfs.append(df_ano)
            except Exception as e:
                print(f"Erro no ano {ano}: {e}")
        
        df_final = pd.concat(dfs, ignore_index=True)

        # Remove a coluna desnecessária, se existir
        if 'Unnamed: 2' in df_final.columns:
            df_final = df_final.drop(columns='Unnamed: 2')

        # Chama a função de categorização
        df_final = self.categorizar(df_final)

        return df_final
    
    def encode_latin1(self, df):
        df['Produto'] = df['Produto'].astype(str).apply(
            lambda x: x.encode('latin1').decode('utf-8') if isinstance(x, str) else x
        )
        return df  # <- Retornar o DataFrame inteiro

    
    def replace_quantidade(self, df):
        df['Quantidade (L.)'] = df['Quantidade (L.)'].replace('-','0')
        return df['Quantidade (L.)'] 

    def column_to_numeric(self, df):
        df['Quantidade (L.)'] = (
            df['Quantidade (L.)']
            .astype(str)
            .str.replace('.', '', regex=False)
            .str.replace('-', '', regex=False)
            .str.strip()
        )

        df['Quantidade (L.)'] = pd.to_numeric(df['Quantidade (L.)'], errors='coerce')
        return df['Quantidade (L.)'] 
        
    def exec(self):
        # 1. Baixar dados
        df = self.table_producao()

        # 2. Corrigir encoding
        df = self.encode_latin1(df)

        # 3. Reformatar quantidade
        df['Quantidade (L.)'] = self.replace_quantidade(df)
        df['Quantidade (L.)']  = self.column_to_numeric(df)

        # 4. Categorizar e remover linhas de categoria
        df = self.categorizar(df)
        df = self.remover_categorias(df)

        # 5. Atualiza o estado interno e retorna
        self.df = df
        return df


In [18]:
run = Scraper()
run.exec()

,Produto,Quantidade (L.),ano,Categoria
0,Tinto,174224052.0,1970,VINHO DE MESA
1,Branco,748400.0,1970,VINHO DE MESA
2,Rosado,42236152.0,1970,VINHO DE MESA
3,Tinto,7591557.0,1970,VINHO FINO DE MESA (VINIFERA)
4,Branco,15562889.0,1970,VINHO FINO DE MESA (VINIFERA)
...,...,...,...,...
2641,Vinho acidificado,2500.0,2023,DERIVADOS
2642,Mosto parcialmente fermentado,0.0,2023,DERIVADOS
2643,Outros derivados,652301.0,2023,DERIVADOS
2644,nan,NaN,2023,DERIVADOS
